In [ ]:
!sudo apt update
!sudo apt install libcairo2-dev \
    texlive texlive-latex-extra texlive-fonts-extra \
    texlive-latex-recommended texlive-science \
    tipa libpango1.0-dev
!pip install manim
!pip install IPython==8.21.0

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://cli.github.com/packages stable InRelease [3,917 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:8 https://cli.github.com/packages stable/main amd64 Packages [355 B]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [85.0 kB]
Get:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease [24.6 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,908 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [6,749 kB]
Ge

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.0/810.0 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.0 MB/s eta 0:00:00
  Attempting uninstall: IPython
    Found existing installation: ipython 7.34.0
    Uninstalling ipython-7.34.0:
      Successfully uninstalled ipython-7.34.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython==7.34.0, but you have ipython 8.21.0 which is incompatible.


In [21]:
# Setup Cell
from manim import *
import numpy as np

# Configure Manim to save output within the Colab environment's /content directory
config.media_dir = 'media'
config.video_dir = 'media/videos'

In [22]:
%%manim -qm -v WARNING ParallelPlatesField
class ParallelPlatesField(Scene):
    def construct(self):
        # We represent the A/2 width plates as a 2D cross section
        plate_width = 4.0
        plate_distance = 2.0

        # Draw the plates
        top_plate = Line(LEFT * plate_width/2, RIGHT * plate_width/2, color=RED, stroke_width=6).shift(UP * plate_distance/2)
        bottom_plate = Line(LEFT * plate_width/2, RIGHT * plate_width/2, color=BLUE, stroke_width=6).shift(DOWN * plate_distance/2)

        # Add +1 and -1 unit charge labels
        plus_signs = VGroup(*[Tex("+", color=RED).scale(0.6).move_to(np.array([x, plate_distance/2 + 0.4, 0])) for x in np.linspace(-plate_width/2, plate_width/2, 7)])
        minus_signs = VGroup(*[Tex("-", color=BLUE).scale(0.6).move_to(np.array([x, -plate_distance/2 - 0.4, 0])) for x in np.linspace(-plate_width/2, plate_width/2, 7)])

        self.play(Create(top_plate), Create(bottom_plate))
        self.play(FadeIn(plus_signs), FadeIn(minus_signs))

        # Function to calculate electric field from discrete charges across the plates
        def calc_efield(pos):
            field = np.zeros(3)
            num_charges = 30 # Discretize the plates into 30 point charges to simulate a continuous sheet

            # Top plate (positive charge +1 spread out)
            for x in np.linspace(-plate_width/2, plate_width/2, num_charges):
                charge_pos = np.array([x, plate_distance/2, 0])
                r = pos - charge_pos
                dist = np.linalg.norm(r)
                if dist > 0.1: # Prevent division by zero at the plate surface
                    field += (r / dist**3)

            # Bottom plate (negative charge -1 spread out)
            for x in np.linspace(-plate_width/2, plate_width/2, num_charges):
                charge_pos = np.array([x, -plate_distance/2, 0])
                r = pos - charge_pos
                dist = np.linalg.norm(r)
                if dist > 0.1:
                    field -= (r / dist**3)

            return field

        # Create the vector field mapping
        vector_field = ArrowVectorField(
            calc_efield,
            x_range=[-6, 6, 0.5],
            y_range=[-4, 4, 0.5],
            length_func=lambda norm: 0.45 * np.tanh(norm)
        )

        self.play(Create(vector_field), run_time=3)
        self.wait(2)

Manim Community v0.19.2

In [ ]:
%%manim -qm -v WARNING ParallelPlates3D
class ParallelPlates3D(ThreeDScene):
    def construct(self):
        # Set initial 3D camera angle
        self.set_camera_orientation(phi=60 * DEGREES, theta=20 * DEGREES)

        plate_width = 4.0
        plate_distance = 2.0

        # Draw the plates as 3D Prisms (Width, Height, Depth)
        top_plate = Prism(dimensions=[plate_width, 0.2, plate_width]).set_color(RED).set_opacity(0.5).shift(UP * plate_distance/2)
        bottom_plate = Prism(dimensions=[plate_width, 0.2, plate_width]).set_color(BLUE).set_opacity(0.5).shift(DOWN * plate_distance/2)

        self.play(Create(top_plate), Create(bottom_plate))

        # Calculate electric field for the center slice (z=0)
        def calc_efield(pos):
            field = np.zeros(3)
            num_charges = 20

            for x in np.linspace(-plate_width/2, plate_width/2, num_charges):
                charge_pos = np.array([x, plate_distance/2, 0])
                r = pos - charge_pos
                dist = np.linalg.norm(r)
                if dist > 0.1:
                    field += (r / dist**3)

            for x in np.linspace(-plate_width/2, plate_width/2, num_charges):
                charge_pos = np.array([x, -plate_distance/2, 0])
                r = pos - charge_pos
                dist = np.linalg.norm(r)
                if dist > 0.1:
                    field -= (r / dist**3)

            return field

        # Plot the 2D slice of the vector field inside the 3D space
        vector_field = ArrowVectorField(
            calc_efield,
            x_range=[-5, 5, 0.75],
            y_range=[-3, 3, 0.75],
            length_func=lambda norm: 0.45 * np.tanh(norm)
        )

        self.play(Create(vector_field), run_time=2)

        # Pan the camera to show the 3D depth
        self.move_camera(phi=45 * DEGREES, theta=80 * DEGREES, run_time=4)
        self.wait(2)

Manim Community v0.19.2

In [ ]:
%%manim -qm -v WARNING CoaxialCylinder3DPlanes
class CoaxialCylinder3DPlanes(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=60 * DEGREES, theta=20 * DEGREES)

        r_a = 0.5
        r_b = 3.0
        h = 6.0

        # Bypass the default Cylinder to guarantee a hollow tube with no caps
        inner_cyl = Surface(
            lambda u, v: np.array([r_a * np.cos(u), r_a * np.sin(u), v]),
            u_range=[0, 2*np.pi],
            v_range=[-h/2, h/2],
            resolution=(24, 24),
            checkerboard_colors=[RED, RED],
            fill_color=RED,
            stroke_color=RED
        ).set_opacity(0.8)

        outer_cyl = Surface(
            lambda u, v: np.array([r_b * np.cos(u), r_b * np.sin(u), v]),
            u_range=[0, 2*np.pi],
            v_range=[-h/2, h/2],
            resolution=(24, 24),
            checkerboard_colors=[BLUE, BLUE],
            fill_color=BLUE,
            stroke_color=BLUE
        ).set_opacity(0.15)

        self.play(Create(outer_cyl), Create(inner_cyl))

        def calc_efield(pos):
            x, y, z = pos
            r = np.sqrt(x**2 + y**2)
            if r < r_a or r > r_b:
                return np.zeros(3)
            magnitude = 1.5 / r
            return magnitude * np.array([x/r, y/r, 0])

        # Generate vectors at z = -2, 0, and 2
        vector_field = ArrowVectorField(
            calc_efield,
            x_range=[-3.5, 3.5, 0.75],
            y_range=[-3.5, 3.5, 0.75],
            z_range=[-2, 2, 2],
            colors=[BLUE, PURPLE, RED],
            length_func=lambda norm: 0.45 * np.tanh(norm)
        )

        self.play(Create(vector_field), run_time=3)
        self.move_camera(phi=75 * DEGREES, theta=70 * DEGREES, run_time=5)
        self.wait(2)

In [ ]:
%%manim -qm -v WARNING SphericalCapacitor3DPlanes
class SphericalCapacitor3DPlanes(ThreeDScene):
    def construct(self):
        self.set_camera_orientation(phi=55 * DEGREES, theta=30 * DEGREES)

        r_a = 0.4
        r_b = 3.0

        # Inner point (positive/RED) and Outer shell (negative/BLUE)
        inner_sph = Sphere(radius=r_a, checkerboard_colors=[RED, RED]).set_opacity(0.9)
        outer_sph = Sphere(radius=r_b, checkerboard_colors=[BLUE, BLUE]).set_opacity(0.1)

        self.play(Create(outer_sph), Create(inner_sph))

        def calc_efield(pos):
            x, y, z = pos
            r = np.sqrt(x**2 + y**2 + z**2)
            if r < r_a or r > r_b:
                return np.zeros(3)
            magnitude = 1.0 / (r**2)
            return magnitude * np.array([x/r, y/r, z/r])

        # Define the three intersecting planes (XY, XZ, YZ)
        vf_xy = ArrowVectorField(calc_efield, x_range=[-3.5, 3.5, 0.75], y_range=[-3.5, 3.5, 0.75], z_range=[0, 0, 1], colors=[BLUE, PURPLE, RED], length_func=lambda norm: 0.45 * np.tanh(norm * 2))
        vf_xz = ArrowVectorField(calc_efield, x_range=[-3.5, 3.5, 0.75], y_range=[0, 0, 1], z_range=[-3.5, 3.5, 0.75], colors=[BLUE, PURPLE, RED], length_func=lambda norm: 0.45 * np.tanh(norm * 2))
        vf_yz = ArrowVectorField(calc_efield, x_range=[0, 0, 1], y_range=[-3.5, 3.5, 0.75], z_range=[-3.5, 3.5, 0.75], colors=[BLUE, PURPLE, RED], length_func=lambda norm: 0.45 * np.tanh(norm * 2))

        vector_field = VGroup(vf_xy, vf_xz, vf_yz)

        self.play(Create(vector_field), run_time=3)
        self.move_camera(phi=75 * DEGREES, theta=80 * DEGREES, run_time=5)
        self.wait(2)

In [ ]:
%%manim -qm -v WARNING CapacitancePipeline
class CapacitancePipeline(Scene):
    def construct(self):
        title = Tex("The 3-Step Capacitance Pipeline", color=YELLOW).to_edge(UP)

        # Step 1: Gauss's Law
        step1_label = Tex("1. Find Electric Field (Gauss's Law):").shift(UP*1.5 + LEFT*2)
        gauss_eq = MathTex(r"\oint \mathbf{E} \cdot d\mathbf{A} = \frac{Q_{enc}}{\varepsilon}").next_to(step1_label, DOWN, aligned_edge=LEFT)

        # Step 2: Electric Potential
        step2_label = Tex("2. Find Voltage (Line Integral):").next_to(gauss_eq, DOWN, buff=0.7).align_to(step1_label, LEFT)
        volt_eq = MathTex(r"V = -\int \mathbf{E} \cdot d\mathbf{l}").next_to(step2_label, DOWN, aligned_edge=LEFT)

        # Step 3: Capacitance
        step3_label = Tex("3. Calculate Capacitance:").next_to(volt_eq, DOWN, buff=0.7).align_to(step1_label, LEFT)
        cap_eq = MathTex(r"C = \frac{Q}{V}").next_to(step3_label, DOWN, aligned_edge=LEFT)

        self.play(Write(title))
        self.play(Write(step1_label), FadeIn(gauss_eq))
        self.wait(1);
        self.play(Write(step2_label), FadeIn(volt_eq))
        self.wait(1);
        self.play(Write(step3_label), FadeIn(cap_eq))
        self.wait(2);

        # Highlight the Q/V relationship
        highlight_box = SurroundingRectangle(cap_eq, color=RED, buff=0.2)
        self.play(Create(highlight_box))
        self.wait(2);

In [ ]:
%%manim -qm -v WARNING CapacitancePipeline
class CapacitancePipeline(Scene):
    def construct(self):
        title = Tex("The 3-Step Capacitance Pipeline", color=YELLOW).to_edge(UP)

        # Set a global scale for all text and equations
        tex_scale = 0.8

        # --- LEFT COLUMN (Steps 1 & 2) ---
        step1_label = Tex("1. Find Electric Field (Gauss's Law):").scale(tex_scale)
        gauss_eq = MathTex(r"\oint \mathbf{E} \cdot d\mathbf{A} = \frac{Q_{enc}}{\varepsilon}").scale(tex_scale)
        step1_group = VGroup(step1_label, gauss_eq).arrange(DOWN, aligned_edge=LEFT)

        step2_label = Tex("2. Find Voltage (Line Integral):").scale(tex_scale)
        volt_eq = MathTex(r"V = -\int \mathbf{E} \cdot d\mathbf{l}").scale(tex_scale)
        step2_group = VGroup(step2_label, volt_eq).arrange(DOWN, aligned_edge=LEFT)

        # Group Steps 1 and 2 together and position them on the left half of the screen
        left_column = VGroup(step1_group, step2_group).arrange(DOWN, aligned_edge=LEFT, buff=1.0)
        left_column.move_to(LEFT * 3.5 + DOWN * 0.5)

        # --- RIGHT COLUMN (Step 3) ---
        step3_label = Tex("3. Calculate Capacitance:").scale(tex_scale)
        cap_eq = MathTex(r"C = \frac{Q}{V}").scale(tex_scale)

        # Position Step 3 on the right half, aligning its top with the top of Step 1
        right_column = VGroup(step3_label, cap_eq).arrange(DOWN, aligned_edge=LEFT)
        right_column.move_to(RIGHT * 3.5).align_to(left_column, UP)

        # Animation Sequence
        self.play(Write(title))
        self.play(Write(step1_label), FadeIn(gauss_eq))
        self.wait(1);
        self.play(Write(step2_label), FadeIn(volt_eq))
        self.wait(1);
        self.play(Write(step3_label), FadeIn(cap_eq))
        self.wait(2);

        # Highlight the Q/V relationship
        highlight_box = SurroundingRectangle(cap_eq, color=RED, buff=0.2)
        self.play(Create(highlight_box))
        self.wait(2);

Manim Community v0.19.2

In [23]:
%%manim -qm -v WARNING ParallelPlateDerivation
class ParallelPlateDerivation(Scene):
    def construct(self):
        title = Tex("Parallel Plate Capacitor", color=YELLOW).to_edge(UP)
        self.add(title)

        # Define fixed anchor points to completely prevent overlapping
        pos_top = UP * 2.2
        pos_mid = ORIGIN
        pos_bot = DOWN * 2.2

        # Step 1: Gauss's Law
        eq1 = MathTex(r"\oint \mathbf{E} \cdot d\mathbf{A}", r"=", r"\frac{Q}{\varepsilon}")
        eq2 = MathTex(r"E \cdot A", r"=", r"\frac{Q}{\varepsilon}")
        eq3 = MathTex(r"E", r"=", r"\frac{Q}{A\varepsilon}")

        # Step 2: Voltage
        eq4 = MathTex(r"V", r"=", r"\int_0^d E \, dx")
        eq5 = MathTex(r"V", r"=", r"E \cdot d")
        eq6 = MathTex(r"V", r"=", r"\frac{Qd}{A\varepsilon}")

        # Step 3: Capacitance
        eq7 = MathTex(r"C", r"=", r"\frac{Q}{V}")
        eq8 = MathTex(r"C", r"=", r"\frac{Q}{\frac{Qd}{A\varepsilon}}")
        eq9 = MathTex(r"C", r"=", r"\frac{\varepsilon A}{d}", color=GREEN).scale(1.2)

        # Animation Sequence - Step 1
        self.play(Write(eq1))
        self.wait(1);
        self.play(ReplacementTransform(eq1, eq2))
        self.wait(1);
        self.play(ReplacementTransform(eq2, eq3))
        self.wait(1);

        # Move Gauss result to the top anchor
        self.play(eq3.animate.move_to(pos_top))

        # Animation Sequence - Step 2 (Draws at the middle anchor)
        eq4.move_to(pos_mid)
        self.play(Write(eq4))
        self.wait(1);
        self.play(ReplacementTransform(eq4, eq5))
        self.wait(1);
        self.play(ReplacementTransform(eq5, eq6))
        self.wait(1);

        # Animation Sequence - Step 3 (Draws at the bottom anchor)
        eq7.move_to(pos_bot)
        eq8.move_to(pos_bot)
        self.play(Write(eq7))
        self.wait(1);
        self.play(ReplacementTransform(eq7, eq8))
        self.wait(1);

        # Show Q cancelling
        cross_q1 = Line(eq8[2][0].get_corner(DL), eq8[2][0].get_corner(UR), color=RED)
        cross_q2 = Line(eq8[2][1:2].get_corner(DL), eq8[2][1:2].get_corner(UR), color=RED)
        self.play(Create(cross_q1), Create(cross_q2))
        self.wait(1);

        # Final Transformation
        self.play(ReplacementTransform(VGroup(eq8, cross_q1, cross_q2), eq9.move_to(pos_bot)))
        self.play(Circumscribe(eq9, color=YELLOW, time_width=2))
        self.wait(2);

Manim Community v0.19.2

In [28]:
%%manim -qm -v WARNING CylinderDerivation

class CylinderDerivation(Scene):
    def construct(self):
        title = Tex("Coaxial Cylinder Capacitor", color=YELLOW).to_edge(UP)
        self.add(title)

        # Define fixed anchor points to utilize the full viewport and prevent smushing
        pos_top = UP * 2.2
        pos_mid = ORIGIN
        pos_bot = DOWN * 2.2

        # Step 1: Gauss's Law
        eq1 = MathTex(r"E \cdot (2\pi r L)", r"=", r"\frac{Q}{\varepsilon}")
        eq2 = MathTex(r"E", r"=", r"\frac{Q}{2\pi \varepsilon L r}")

        # Step 2: Voltage (Integration of 1/r results in ln)
        eq3 = MathTex(r"V", r"=", r"\int_a^b \frac{Q}{2\pi \varepsilon L r} \, dr")
        eq4 = MathTex(r"V", r"=", r"\frac{Q}{2\pi \varepsilon L} \ln\left(\frac{b}{a}\right)")

        # Step 3: Capacitance (The ratio Q/V)
        eq5 = MathTex(r"C", r"=", r"\frac{Q}{V}")
        eq6 = MathTex(r"C", r"=", r"\frac{Q}{\frac{Q}{2\pi \varepsilon L} \ln\left(\frac{b}{a}\right)}")
        eq7 = MathTex(r"C", r"=", r"\frac{2\pi \varepsilon L}{\ln\left(\frac{b}{a}\right)}", color=GREEN).scale(1.1)

        # Animation Sequence - Step 1 (Gauss)
        self.play(Write(eq1))
        self.wait(1)
        self.play(ReplacementTransform(eq1, eq2))
        self.wait(1)

        # Move Gauss result to the top anchor
        self.play(eq2.animate.move_to(pos_top))

        # Animation Sequence - Step 2 (Voltage - at the middle anchor)

        eq3.move_to(pos_mid)
        self.play(Write(eq3))
        self.wait(1)
        self.play(ReplacementTransform(eq3, eq4))
        self.wait(1)

        # Animation Sequence - Step 3 (Capacitance - at the bottom anchor)
        eq5.move_to(pos_bot)
        self.play(Write(eq5))
        self.wait(1)
        self.play(ReplacementTransform(eq5, eq6.move_to(pos_bot)))
        self.wait(1)

        # Final Transformation and Highlight
        self.play(ReplacementTransform(eq6, eq7.move_to(pos_bot)))
        self.play(Circumscribe(eq7, color=YELLOW))
        self.wait(2)

Manim Community v0.19.2

In [24]:
%%manim -qm -v WARNING SphereDerivation

class SphereDerivation(Scene):
    def construct(self):
        title = Tex("Spherical Capacitor", color=YELLOW).to_edge(UP)
        self.add(title)

        # Define fixed anchor points to prevent jumbling
        pos_top = UP * 2.2
        pos_mid = ORIGIN
        pos_bot = DOWN * 2.2

        # Step 1: Gauss's Law
        eq1 = MathTex(r"E \cdot (4\pi r^2)", r"=", r"\frac{Q}{\varepsilon}")
        eq2 = MathTex(r"E", r"=", r"\frac{Q}{4\pi \varepsilon r^2}")

        # Step 2: Voltage
        eq3 = MathTex(r"V", r"=", r"\int_a^b \frac{Q}{4\pi \varepsilon r^2} \, dr")
        eq4 = MathTex(r"V", r"=", r"\frac{Q}{4\pi \varepsilon} \left[ -\frac{1}{r} \right]_a^b")
        eq5 = MathTex(r"V", r"=", r"\frac{Q}{4\pi \varepsilon} \left( \frac{1}{a} - \frac{1}{b} \right)")
        eq6 = MathTex(r"V", r"=", r"\frac{Q(b-a)}{4\pi \varepsilon a b}")

        # Step 3: Capacitance
        eq7 = MathTex(r"C", r"=", r"\frac{Q}{V}")
        eq8 = MathTex(r"C", r"=", r"\frac{4\pi \varepsilon a b}{b-a}", color=GREEN).scale(1.1)

        # Animation Sequence - Step 1 (Gauss)
        self.play(Write(eq1))
        self.wait(1)
        self.play(ReplacementTransform(eq1, eq2))
        self.wait(1)

        # Move Gauss result to the top anchor to clear space
        self.play(eq2.animate.move_to(pos_top))

        # Animation Sequence - Step 2 (Voltage - at the middle anchor)
        eq3.move_to(pos_mid)
        self.play(Write(eq3))
        self.wait(1)
        self.play(ReplacementTransform(eq3, eq4))
        self.wait(1)
        self.play(ReplacementTransform(eq4, eq5))
        self.wait(1)
        self.play(ReplacementTransform(eq5, eq6))
        self.wait(1)

        # Animation Sequence - Step 3 (Capacitance - at the bottom anchor)
        eq7.move_to(pos_bot)
        self.play(Write(eq7))
        self.wait(1)
        self.play(ReplacementTransform(eq7, eq8.move_to(pos_bot)))

        # Final Highlight
        self.play(Circumscribe(eq8, color=YELLOW))
        self.wait(2)

Manim Community v0.19.2

In [ ]:
%%manim -qm -v WARNING CapacitanceComparisonFinal

from manim import *
import numpy as np

class CapacitanceComparisonFinal(Scene):
    def construct(self):
        title = Tex("Capacitance vs. Separation Distance ($d$)", color=YELLOW).to_edge(UP)
        subtitle = Tex("Normalized Material Budget (Area $\\approx 1$ $m^2$)", font_size=24).next_to(title, DOWN)

        # Physical constants
        epsilon = 1.0
        # Cleaned outer radius
        b = 0.3
        y_max_clamp = 10.0

        axes = Axes(
            x_range=[0.01, 0.25, 0.05],
            y_range=[0, y_max_clamp, 2],
            x_length=8,
            y_length=5,
            axis_config={"include_tip": True, "numbers_to_exclude": [0]},
            tips=False
        ).shift(DOWN*0.5).add_coordinates(font_size=20)

        labels = axes.get_axis_labels(
            x_label="Separation (d)",
            y_label="Capacitance (C)"
        )

        # --- 1. Parallel Plate (White Group) ---
        graph_pp = axes.plot(
            lambda x: min((0.5 * epsilon) / x, y_max_clamp),
            x_range=[0.02, 0.25],
            color=WHITE
        )
        label_pp = Tex("Parallel Plate", color=WHITE, font_size=22)
        eq_pp = MathTex(r"C \approx \frac{0.5}{d}", color=WHITE, font_size=22)
        group_pp = VGroup(label_pp, eq_pp).arrange(DOWN, aligned_edge=LEFT, buff=0.1)
        # Positioned up and to the right
        group_pp.move_to(axes.c2p(0.28, 3.2))

        # --- 2. Cylindrical (Blue Group) ---
        # If Area = 1, then 2*pi*b*L = 1 => 2*pi*L = 1/b = 3.33
        graph_cyl = axes.plot(
            lambda x: min((3.33 * epsilon) / np.log(b / (b - x)), y_max_clamp) if x < b - 0.01 else 0,
            x_range=[0.02, 0.23],
            color=BLUE
        )
        label_cyl = Tex("Cylindrical", color=BLUE, font_size=22)
        eq_cyl = MathTex(r"C \approx \frac{3.33}{\ln\left(\frac{0.3}{0.3-d}\right)}", color=BLUE, font_size=22)
        group_cyl = VGroup(label_cyl, eq_cyl).arrange(DOWN, aligned_edge=LEFT, buff=0.1)
        # Positioned further up and right for clarity
        group_cyl.move_to(axes.c2p(0.20, 5.8))

        # --- 3. Spherical (Red Group) ---
        # If Area = 1, then C = (b-d)/bd = (0.3-d)/0.3d
        graph_sph = axes.plot(
            lambda x: min((epsilon * (b - x)) / (b * x), y_max_clamp) if x < b else 0,
            x_range=[0.02, 0.25],
            color=RED
        )
        label_sph = Tex("Spherical", color=RED, font_size=22)
        eq_sph = MathTex(r"C \approx \frac{0.3-d}{0.3d}", color=RED, font_size=22)
        group_sph = VGroup(label_sph, eq_sph).arrange(DOWN, aligned_edge=LEFT, buff=0.1)
        # Positioned to the right of the curve
        group_sph.move_to(axes.c2p(0.14, 8.5))

        # --- Animation Sequence ---
        self.play(Write(title), Write(subtitle))
        self.play(Create(axes), Write(labels))

        self.play(Create(graph_pp), Write(group_pp), run_time=1.5)
        self.play(Create(graph_cyl), Write(group_cyl), run_time=1.5)
        self.play(Create(graph_sph), Write(group_sph), run_time=1.5)

        self.wait(3)

Manim Community v0.19.2